Source: https://python.langchain.com/v0.1/docs/use_cases/tool_use/prompting/ 

In [2]:

from langchain import hub
from langchain_community.chat_models import ChatOllama
from langgraph.graph import END, StateGraph
from typing import Dict, TypedDict
from langchain.prompts import PromptTemplate
import pprint
import os

ModuleNotFoundError: No module named 'httpx'

In [ ]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(
    api_key="ollama",
    model="elvee/hermes-2-pro-llama-3:8b-Q5_K_M",
    base_url="http://localhost:11434/v1",
    temperature=0
)

In [ ]:
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

In [ ]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiply two integers together.
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}


In [ ]:
multiply.invoke({"first_int": 4, "second_int": 5})

20

In [ ]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description([multiply])
rendered_tools

'multiply(first_int: int, second_int: int) -> int - Multiply two integers together.'

For example: 
{'name': 'multiply', 'arguments': {'arg1': value_1, 'arg2': value_2}}

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool: 

""" + rendered_tools + """

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.
For example: 
{{'name': 'multiply', 'arguments': {{'arg1': value_1, 'arg2': value_2}}}}


"""

print(system_prompt)

prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool: 

multiply(first_int: int, second_int: int) -> int - Multiply two integers together.

Given the user input, return the name and input of the tool to use. 
Return your response as a JSON blob with 'name' and 'arguments' keys.
For example: 
{{'name': 'multiply', 'arguments': {{'arg1': value_1, 'arg2': value_2}}}}





In [ ]:
from langchain_core.output_parsers import JsonOutputParser


chain = prompt | llm | JsonOutputParser()
chain.invoke({"input": "what's thirteen times 4"})

{'name': 'multiply', 'arguments': {'first_int': 13, 'second_int': 4}}

In [ ]:
from operator import itemgetter

chain = prompt | llm | JsonOutputParser() | itemgetter("arguments") | multiply
chain.invoke({"input": "what's thirteen times 4"})

52